In [1]:
import pandas as pd
import os
from PIL import Image


In [3]:
csv_path = 'data/sample-dayClip6/sample-dayClip6/frameAnnotationsBOX.csv'
images_dir = 'data_arman/valid/images/'

In [4]:
output_labels_dir = 'data_arman/valid/labels/'

In [5]:
os.makedirs(output_labels_dir, exist_ok=True)

In [6]:
df = pd.read_csv(csv_path, sep=';')

In [7]:
def convert_bbox_to_yolo(image_width, image_height, x_min, y_min, x_max, y_max):
    x_center = (x_min + x_max) / 2 / image_width
    y_center = (y_min + y_max) / 2 / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    return x_center, y_center, width, height

In [8]:
category_mapping = {'go': 0,
 'stop': 1,
 'stopLeft': 2,
 'warning': 3,
 'goLeft': 4,
 'warningLeft': 5}

In [10]:
image_dir = 'data_arman/valid/images/'
for image_file in os.listdir(image_dir):
    
    if image_file.endswith(('.jpg', '.png')):  # Adjust extensions if needed
        label_file = os.path.join(output_labels_dir, image_file.rsplit('.', 1)[0] + '.txt')
        open(label_file, 'w').close()  # Create an empty .txt file


In [15]:
for index, row in df.iterrows():
    # Extract data
    filename = row['Filename'].split('/')[1]
    class_name = row['Annotation tag']
    x_min = row['Upper left corner X']
    y_min = row['Upper left corner Y']
    x_max = row['Lower right corner X']
    y_max = row['Lower right corner Y']
    
    # Get image dimensions
    image_path = os.path.join(images_dir, filename)
    with Image.open(image_path) as img:
        image_width, image_height = img.size

    # Convert bbox to YOLO format
    x_center, y_center, width, height = convert_bbox_to_yolo(image_width, image_height, x_min, y_min, x_max, y_max)

    # Get class ID (you need to map `class_name` to an integer ID)
    class_id = category_mapping[class_name]  # Replace this with your actual class mapping logic

    # Prepare YOLO annotation line
    yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"

    # Write to corresponding `.txt` file
    label_file_path = os.path.join(output_labels_dir, filename.replace('.jpg', '.txt').replace('.png', '.txt'))
    with open(label_file_path, 'a') as f:  # Append if multiple objects per image
        f.write(yolo_line)

In [20]:
yaml_content = """
train: /data_arman/train/train/
val: /data_arman/valid/images/

nc: 5  # Number of classes
names: ['stop', 'go', 'warning', 'warningLeft', 'stopLeft']  # Class names
"""

with open('data_arman/dataset.yaml', 'w') as f:
    f.write(yaml_content)


In [14]:
row['Filename'].split('/')[1]

'dayClip6--00000.jpg'